In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2022, 11, 22)

### Tables in the process

In [2]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}', 
    
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    }

In [3]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,BH,207,2022-11-18,500,210.00,1000,1215,2022-11-22,220.00,227.70,15.94,"109,756.36",4,"4,523.80",4.30%,392.27%,0,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [4]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = sells['name']
name = names.to_string(index=False)

sr_qty = sells['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = sells['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = sells['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = sells['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = 1215



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,BH,1215,207,2022-11-18,500,210.00,217.35,15.21,Sold,"105,232.56",DTD,None


In [5]:
transaction, name, qty, sell_price, buy_price

('S', 'BH', -500, 220.0, 210.0)

### In case of buy, retreive buy id from latest buy record (id)

In [4]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,RCL,1217,192,2022-11-22,"3,000",31.50,195.62,13.69,Active,"94,709.31",HD,None


In [5]:
transaction = 'B'

names = buys['name']
name = names.to_string(index=False)

sr_qty = buys['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = buys['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

transaction, name, qty, buy_price

('B', 'RCL', 3000, 31.5)

## Begin of Sale transaction

### Begin of Update buy table in MySQL stock database from sale transaction

In [6]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'BH'



,name,date,shares,price,active,period,grade
0,BH,2022-11-18,500,210.00,1,3,A3


In [7]:
new_cost = 210
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + -500, price = 210
WHERE name = 'BH'



In [8]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [9]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'BH'



,name,date,shares,price,active,period,grade
0,BH,2022-11-18,0,210.00,1,3,A3


### If shares become zero and want to delete from buy table

In [10]:
sqlDel = """
DELETE FROM buy
WHERE name = '%s' AND volbuy = 0"""
sqlDel = sqlDel % name
print(sqlDel)


DELETE FROM buy
WHERE name = 'BH' AND volbuy = 0


In [11]:
rp = const.execute(sqlDel)
rp.rowcount

1

### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [12]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'BH'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,BH,2.0500,0.0000,1.1500,0.0000,3.2000,500,2023-03-11,2023-05-10,,0


In [13]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + -500
WHERE name = 'BH'



In [14]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [15]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'BH'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,BH,2.0500,0.0000,1.1500,0.0000,3.2000,0,2023-03-11,2023-05-10,,0


### If shares become zero and want to delete from DIVIDEND table

In [16]:
sqlDel = """
DELETE FROM dividend
WHERE name = '%s'
"""
sqlDel = sqlDel % (name)
print(sqlDel)


DELETE FROM dividend
WHERE name = 'BH'



In [17]:
rp = const.execute(sqlDel)
rp.rowcount

1

### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [18]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'BH'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9061,BH,175.00,116.50,I,210.00,220.00,480.52,0.43,210.00,500,-4,4,500,0,0,5pct,SET50


In [19]:
buy_target  = 210  # must calculate manually beforehand
sell_target = 220
status = 'O'
reason = "5pct"
sell_qty = 500

sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s,
qty = %s, buy_target = %s, sell_target = %s, reason = '%s', status = '%s'
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, sell_qty, buy_target, sell_target, reason, status, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + -500,
qty = 500, buy_target = 210, sell_target = 220, reason = '5pct', status = 'O'
WHERE name = 'BH'



In [20]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [21]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'BH'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9061,BH,175.00,116.50,O,210.00,220.00,480.52,0.43,210.00,500,-4,4,0,0,0,5pct,SET50


### End of Update stocks table in SQLite port_lite database from sale transaction

### If shares become zero and no longer wants to keep STOCK record

In [ ]:
sqlDel = """
DELETE FROM stocks
WHERE name = '%s' AND available_qty = 0
"""
sqlDel = sqlDel % name
print(sqlDel)

In [ ]:
rp = conlite.execute(sqlDel)
rp.rowcount

## End of Sale process

## Begin of  Buy process

### Begin of Update buy table in MySQL database from buy transaction

In [6]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy['date'] = pd.to_datetime(buy['date'])
buy.drop(['volsell', 'volbal'], axis=1, inplace = True)
buy.rename(columns={'volbuy':'shares'}, inplace = True)
buy['shares'] = buy['shares'].astype('int64')
buy


SELECT * 
FROM buy 
WHERE name = 'RCL'



,name,date,shares,price,active,dividend,period,grade
0,RCL,2021-11-08,21000,41.0,1,7.0,2,A1


### In case there is the same stock name in port

In [7]:
new_cost = 39.80 # new average cost must be calculated manually beforehand
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + 3000, price = 39.8
WHERE name = 'RCL'



In [8]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [9]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'RCL'



,name,date,shares,price,active,dividend,period,grade
0,RCL,2021-11-08,"24,000",39.80,1,7.0000,2,A1


### In case of totally new stock

In [ ]:
rcd = ['BH', '2022-11-18', 500, 210, 0, 0, 1, 0, 3,'A3']
rcd

In [ ]:
sqlIns = """
INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)

In [ ]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

In [ ]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)

### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [10]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'RCL' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,RCL,3.0000,2.2500,1.7500,0.0000,7.0000,"21,000",2022-11-24,2022-12-09,,1


In [11]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + 3000
WHERE name = 'RCL'



In [12]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [13]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'RCL' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,RCL,3.0000,2.2500,1.7500,0.0000,7.0000,"24,000",2022-11-24,2022-12-09,,1


### In case of totally new stock

In [ ]:
rcd = ['BH', 2.05, 0, 1.15, 0, 3.20, 500, '2023-03-11', '2023-05-10', '', 0]
rcd

In [ ]:
sqlIns = """
INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)

In [ ]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

### End of Create dividend table in MySQL stock database from buy transaction

### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [14]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks


SELECT * 
FROM stocks 
WHERE name = 'RCL'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,662,RCL,67.75,30.5,B,31.5,41.25,407.01,1.22,41,3000,-16,26,21000,0,0,RD40pct,SET100


In [15]:
new_cost = 39.80
buy_target = 30.25
sell_target = 40.00
status = 'I'
reason = '5pct'
new_buy_qty = 3000
new_sell_qty = 3000
new_cost, buy_target, sell_target, status, reason, new_buy_qty, new_sell_qty 

(39.8, 30.25, 40.0, 'I', '5pct', 3000, 3000)

In [16]:
sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s, 
cost = %s, buy_target = %s, sell_target = %s, status = '%s', 
reason = '%s', qty = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, buy_target, sell_target, status, reason, new_sell_qty, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + 3000, 
cost = 39.8, buy_target = 30.25, sell_target = 40.0, status = 'I', 
reason = '5pct', qty = 3000
WHERE name = 'RCL'



In [17]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [18]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'RCL'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,662,RCL,67.75,30.50,I,30.25,40.00,407.01,1.22,39.80,"3,000",-16,26,"24,000",0,0,5pct,SET100


### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy process